In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
pd.set_option('display.max_columns',None) # To display all columns
pd.set_option('display.max_rows',None) # To display all columns
import feather
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import numpy as np
import pandas as pd
import glob
import warnings
warnings.filterwarnings('ignore')
from functools import reduce

In [2]:
questions_df = pd.read_csv('questions.csv')
questions_df.bundle_id = questions_df.bundle_id.str.slice(start=1)
questions_df.explanation_id = questions_df.explanation_id.str.slice(start=1)
questions_df.deployed_at = pd.to_datetime(questions_df.deployed_at, unit='ms')
dfq = questions_df[['question_id', 'correct_answer', 'bundle_id', 'explanation_id' , 'part', 'tags', 'deployed_at']]
dfq.head()

,question_id,correct_answer,bundle_id,explanation_id,part,tags,deployed_at
0,q1,b,1,1,1,1;2;179;181,2019-05-17 11:40:17.098
1,q2,a,2,2,1,15;2;182,2019-05-17 11:40:19.720
2,q3,b,3,3,1,14;2;179;183,2019-05-17 11:40:22.784
3,q4,b,4,4,1,9;2;179;184,2019-05-17 11:40:25.357
4,q5,c,5,5,1,8;2;179;181,2019-05-17 11:40:28.439


In [3]:
KT3 = pd.read_feather('KT3.ftr')
KT3.dropna(axis = 0, how = 'all', inplace = True)

In [4]:
# subset df for notnull question_id rows only (only work with rows that have questions)
KT3 = KT3[KT3['item_id'].notnull()]

In [5]:
K3_userid_counts = KT3.user_iD.value_counts()
mask = K3_userid_counts >= 30
KT3_userids = mask[mask].index
K3 = KT3[KT3.user_iD.isin(KT3_userids)]

K3.head()

,timestamp,action_type,item_id,source,user_answer,platform,user_iD
0,1565096151269,enter,b3544,diagnosis,None,mobile,1
1,1565096187972,respond,q5012,diagnosis,b,mobile,1
2,1565096194904,submit,b3544,diagnosis,None,mobile,1
3,1565096195001,enter,b3238,diagnosis,None,mobile,1
4,1565096218682,respond,q4706,diagnosis,c,mobile,1


In [ ]:
KT1 = pd.read_feather('DEMOKT1.ftr')
questions_df = pd.read_csv('questions.csv')

questions_df.bundle_id = questions_df.bundle_id.str.slice(start=1)
questions_df.explanation_id = questions_df.explanation_id.str.slice(start=1)
questions_df.deployed_at = pd.to_datetime(questions_df.deployed_at, unit='ms')
dfq = questions_df[['question_id', 'correct_answer', 'bundle_id', 'explanation_id' , 'part', 'tags', 'deployed_at']]
#dfq.head()


# subset df for notnull question_id rows only (only work with rows that have questions)
df2 = KT1[KT1['question_id'].notnull()]

# left join student dataframe and answer key dataframe on the 'question_id' column 
dfm = pd.merge(df2, dfq,  how='left', left_on=['question_id'], right_on = ['question_id'])

In [ ]:
# create a 'correct' column
dfm['correct'] = np.nan

# loop - if user answers == correct answer, then dfq['correct'][_]= 1
length_df = dfm['user_iD'].count()
count = 0
for i in range(length_df):
    if dfm['user_answer'][count] == dfm['correct_answer'][count]:
        dfm['correct'][count] = 1
    if dfm['user_answer'][count] != dfm['correct_answer'][count]:
        dfm['correct'][count] = 0
    count+=1